In [14]:
import torch
import torch.nn as nn

# Define the input and output dimensions
input_dim = 256
output_dim = 256

# Define the number of attention heads and the hidden dimension
num_heads = 8
hidden_dim = 128

# Define the dropout rate
dropout = 0.1

# Define the number of layers
num_layers = 12

# Define the decoder layers
decoder_layers = nn.ModuleList([
    nn.TransformerDecoderLayer(d_model=output_dim, nhead=num_heads, dim_feedforward=hidden_dim, dropout=dropout)
    for _ in range(num_layers)
])

# Define the transformer decoder
class TransformerDecoder(nn.Module):
    def __init__(self, layers):
        super(TransformerDecoder, self).__init__()
        self.layers = layers

    def forward(self, input, output, tgt_mask):
        # Reshape the mask tensor to have shape (batch_size, num_heads, seq_len, seq_len)
        attn_mask = tgt_mask.unsqueeze(1).unsqueeze(2)
        print(attn_mask.shape)
        print(input.shape)
        # Pass the input and output through the decoder layers
        for layer in self.layers:
            output = layer(input, output, tgt_mask=attn_mask)
        return output

decoder = TransformerDecoder(decoder_layers)

# Define the input and output tensors
input_tensor = torch.randn(1, 10, input_dim)
output_tensor = torch.randn(1, 15, output_dim)

# Mask the input tensor to prevent the decoder from attending to future tokens
mask = (torch.triu(torch.ones(10, 15)) == 1).transpose(0, 1)
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
print(mask.shape)
# Pass the input and output tensors through the decoder
output = decoder(input_tensor, output_tensor, tgt_mask=mask)


torch.Size([15, 10])
torch.Size([15, 1, 1, 10])
torch.Size([1, 10, 256])


AssertionError: For batched (3-D) `query`, expected `attn_mask` to be `None`, 2-D or 3-D but found 4-D tensor instead

In [10]:
(torch.triu(torch.ones(10, 15)) == 1).transpose(0, 1).float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

tensor([[-inf, 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [-inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0.],
        [-inf, -inf, -inf, 0., 0., 0., 0., 0., 0., 0.],
        [-inf, -inf, -inf, -inf, 0., 0., 0., 0., 0., 0.],
        [-inf, -inf, -inf, -inf, -inf, 0., 0., 0., 0., 0.],
        [-inf, -inf, -inf, -inf, -inf, -inf, 0., 0., 0., 0.],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, 0., 0., 0.],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, 0., 0.],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, 0.],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]])